# TEST of custom Multi-Agent SAC agent on citylearn env

In [1]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import torch

import numpy as np
import pandas as pd

from citylearn.citylearn import CityLearnEnv

from custom_agent.custom_SAC.ma_sac_agents import Agents

from custom_reward.custom_reward import CustomReward

ModuleNotFoundError: No module named 'custom_agent.custom_SAC.ma_sac_agents2'

In [ ]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}
    
def makeEnv(schema_path, reward_function):
    # create environment
    env = CityLearnEnv(schema = schema_path, reward_function = reward_function, central_agent=False)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env


In [ ]:
schema_path = "data/schema.json"

env, wrapper_env = makeEnv(schema_path, CustomReward)

# wrap environment for use in stablebaselines3
# env = NormalizedObservationWrapper(env)
# env = StableBaselines3Wrapper(env)

In [ ]:
sac_agent = Agents(env, batch_size=100, buffer_max_size=100000)

/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


obs 90
act 9


In [ ]:
logs_list = []

for i in range(3):
    # make agent
    sac_agent = Agents(env, batch_size=100, buffer_max_size=100000)
    
    # training run
    logs = sac_agent.train(nr_steps = 720*2, warmup_steps = 720, learn_delay = 100, learn_freq = 1, learn_weight = 1)
    # display(logs)
    
    # save training logs for this run
    logs_list.append(logs)

# print(logs_list.shape)

obs 90
act 9


[Episode 1 mean reward: [-1.05858301 -0.94618482 -0.96865855]] ~ :  51%|█████     | 732/1440 [01:38<01:34,  7.46it/s]


AssertionError: negative electricity consumption for cooling demand | timestep: 14, building: Building_1, device_output: -9.344993591308594, electricity_consumption: -4.1051105764168625